In [1]:
from coverage import *
from sgp4.api import Satrec, WGS72
from skyfield.api import load, EarthSatellite
import folium
import numpy as np

import dask_geopandas

In [2]:
delta_mos = np.arange(0, 361, 22.5)

# <EarthSatellite catalog #39084 epoch 2021-07-18 22:54:21 UTC>
times = gen_times(start_yr=2021, start_mo=7, start_day=18, days=1, step_min=22/60)
inst = camera_model(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800) 

Propogation time: 
 2021-07-18 00:00:00+00:00 
to 
 2021-07-18 23:59:54+00:00

In [3]:
degtorad = (np.pi/180)
# delta_mos = np.array([0., 90., 180., 270.])*degtorad
delta_mos = np.arange(0,360,22.5)*degtorad

In [4]:
delta_mos

array([0.        , 0.39269908, 0.78539816, 1.17809725, 1.57079633,
       1.96349541, 2.35619449, 2.74889357, 3.14159265, 3.53429174,
       3.92699082, 4.3196899 , 4.71238898, 5.10508806, 5.49778714,
       5.89048623])

In [5]:
sats = []
ts = load.timescale()

# for delta_mo in delta_mos:
for delta_mo in delta_mos:
    sat_ref = load.tle_file("./historical_tles/sat39084-tle1.txt")[0]
    satrec = Satrec()
    satrec.sgp4init(
        WGS72,           # gravity model
        'i',             # 'a' = old AFSPC mode, 'i' = improved mode
        sat_ref.model.satnum,               # satnum: Satellite number
        (50 + sat_ref.model.epochyr)*365.5 + sat_ref.model.epochdays,       # epoch: days since 1949 December 31 00:00 UT
        sat_ref.model.bstar,      # bstar: drag coefficient (/earth radii)
        sat_ref.model.nddot, # ndot: ballistic coefficient (revs/day)
        0.0,             # nddot: second derivative of mean motion (revs/day^3)
        sat_ref.model.ecco,       # ecco: eccentricity
        sat_ref.model.argpo, # argpo: argument of perigee (radians)
        sat_ref.model.inclo, # inclo: inclination (radians)
        sat_ref.model.mo + delta_mo, # mo: mean anomaly (radians)
        sat_ref.model.no_kozai, # no_kozai: mean motion (radians/minute)
        sat_ref.model.nodeo, # nodeo: right ascension of ascending node (radians)
    )
    sat_out = EarthSatellite.from_satrec(satrec, ts)
    sats.append(sat_out)

In [6]:
sats[0]

<EarthSatellite catalog #39084 epoch 2021-07-18 22:54:21 UTC>

In [7]:
gdfs = []
for sat in sats:
    fov_df = forecast_fovs(sat, times, inst)
    gdfs.append(fov_df)

fov_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")

fov_df = dask_geopandas.from_geopandas(fov_df, npartitions=4)

## Filter shapes crossing anti-meridian - also in main function
## TODO: Switch to stactools solution
fov_df["lonspan"] = fov_df.bounds['maxx'] - fov_df.bounds['minx']
fov_df = fov_df[fov_df["lonspan"] < 20].copy()

## Create cmap for unique satellites and create color column
sat_ids = list(fov_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
fov_df['color'] = fov_df['id'].apply(cmap)

# fov_df.to_file("./tmp/fovs.geojson")
fov_df

/Users/kevinlalli/miniconda3/envs/coverage/lib/python3.8/site-packages/dask/dataframe/core.py:3956: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('id', 'object'))

  warnings.warn(meta_warning(meta))


,geometry,satellite,id,time,lonspan,color
npartitions=4,,,,,,
0,geometry,object,int64,object,float64,object
15712,...,...,...,...,...,...
31424,...,...,...,...,...,...
47136,...,...,...,...,...,...
62847,...,...,...,...,...,...


In [8]:
## Coverage data analysis for single satellite/ batch of satellites

# aoi = gpd.read_file('./aois/eastern_us.geojson').geometry
# from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

## Filter by AOI - should pass to forecast function instead?
# aoi =  world[world.name == "Venezuela"].geometry
aoi =  world[world.continent == "South America"].geometry

## Plotting FOVs

## Crop the df TODO: make function for various spatial trims
xmin, ymin, xmax, ymax= aoi.total_bounds
fov_df = fov_df.cx[xmin: xmax, ymin: ymax]

## Make a folium map
m = fov_df.explore(color="color", tooltip=["satellite", "time"])

## Add WRS2
wrs2 = gpd.read_file('./WRS2_descending_0/WRS2_descending.shp')
wrs2 = wrs2.cx[xmin: xmax, ymin: ymax]

folium.GeoJson(data=wrs2["geometry"], overlay=False).add_to(m)

## View or save
# m#.save("./tmp/fovs_map.html")

NotImplementedError: Not yet implemented if the GeoDataFrame has no known spatial partitions (you can call the 'calculate_spatial_partitions' method to set it)

In [ ]:
## Calculate revisits on fixed grid

xcell_size = ycell_size = 0.1
grid, grid_shape = calculate_revisits(fov_df, aoi, grid_x=xcell_size, grid_y=ycell_size)
# grid = dask_geopandas.from_geopandas(grid, npartitions=4)
revisit_map(grid, grid_shape, grid_x=xcell_size, grid_y=ycell_size)